In [1]:
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import torchio as tio
import nibabel as nib
import SimpleITK as sitk
from scipy.stats import iqr
from scipy.stats import f_oneway

import os
import json
import numpy as np
import copy
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
# import nipype.interfaces.ants as npants
import ants

this_path = Path(os.getcwd())

In [ ]:
%%bash
export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=2

In [2]:
md_df = pd.read_csv('/mrhome/alejandrocu/Documents/parkinson_classification/bids_3t.csv')
md_df_hc = md_df[md_df['group'] == 0]
md_df_pd = md_df[md_df['group'] == 1]

subj_ids = md_df['id'].values
subj_ids = np.delete(subj_ids, np.where(subj_ids == 'sub-058'))

subj_paths = []
for idx, subj in enumerate(subj_ids):
    base_path = Path(f'/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/{subj}/Results/brain_masked')
    subj_paths.append(str(base_path / f'{subj}_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii'))
    if idx > 1:
        break
print(subj_paths, len(subj_paths))

mov_img_path = Path('/mnt/projects/7TPD/bids/derivatives/hMRI_acu/mni_PD25_20170213_nifti/PD25-T1MPRAGE-template-1mm.nii')
atlas_img_path = Path('/mnt/projects/7TPD/bids/derivatives/hMRI_acu/mni_PD25_20170213_nifti/PD25-subcortical-1mm.nii')
atls_img = ants.image_read(str(atlas_img_path))

['/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/sub-003/Results/brain_masked/sub-003_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii', '/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/sub-004/Results/brain_masked/sub-004_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii', '/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/sub-005/Results/brain_masked/sub-005_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii'] 3


In [3]:
fi = ants.image_read(str(mov_img_path))
mi = ants.image_read(subj_paths[0])
out_path = f'/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/sub-003/Results/antsreg/test'
mytx = ants.registration(fixed=fi, moving=mi, type_of_transform = 'SyNRA', outprefix= out_path)


In [5]:
warped_image = ants.apply_transforms(fixed=fi, moving=mi, transformlist=mytx['fwdtransforms'])
ants.image_write(warped_image, f'{out_path}_warped.nii')

In [6]:
# warping atlas
warped_atlas = ants.apply_transforms(fixed=mi, moving=atls_img, transformlist=mytx['invtransforms'], interpolator='genericLabel')
ants.image_write(warped_atlas, f'{out_path}_warped_atlas.nii')

In [12]:
# full pipe loop
md_df = pd.read_csv('/mrhome/alejandrocu/Documents/parkinson_classification/bids_3t.csv')
md_df_hc = md_df[md_df['group'] == 0]
md_df_pd = md_df[md_df['group'] == 1]

subj_ids = md_df['id'].values
subj_ids = np.delete(subj_ids, np.where(subj_ids == 'sub-058'))

pd25template_path = Path('/mnt/projects/7TPD/bids/derivatives/hMRI_acu/mni_PD25_20170213_nifti/PD25-T1MPRAGE-template-1mm.nii')
pdlabels_path = Path('/mnt/projects/7TPD/bids/derivatives/hMRI_acu/mni_PD25_20170213_nifti/PD25-subcortical-1mm.nii')
pdlabels_img = ants.image_read(str(pdlabels_path))

exp_name = 'synra_CC'
for idx, subj in enumerate(subj_ids):
# subj = 'sub-015'
    print(subj)
    base_path = Path(f'/mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/{subj}/Results/brain_masked')
    r1_map_path = base_path / f'{subj}_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii'
    fi = ants.image_read(str(pd25template_path))
    mi = ants.image_read(str(r1_map_path))
    out_path = base_path.parent / 'antsreg'
    out_path.mkdir(parents=True, exist_ok=True)
    mytx = ants.registration(fixed=fi, moving=mi,
                             type_of_transform = 'SyNRA', #'antsRegistrationSyNQuick[s]',
                             outprefix= str(out_path / exp_name),
                             syn_metric = 'CC')
    save_dict = {'fwdtransforms': mytx['fwdtransforms'], 'invtransforms': mytx['invtransforms']}
    with open(str(out_path / f'{exp_name}_outs.json'), 'w') as f:
        json.dump(save_dict, f)
    # save warped images
    warped_image = ants.apply_transforms(fixed=fi, moving=mi, transformlist=mytx['fwdtransforms'])
    ants.image_write(warped_image, str(out_path / f'{exp_name}_r1topd25_warped.nii'))
    warped_image = ants.apply_transforms(fixed=mi, moving=fi, transformlist=mytx['invtransforms'])
    ants.image_write(warped_image, str(out_path / f'{exp_name}_pd25tor1_warped.nii'))
    # warp atlas
    warped_atlas = ants.apply_transforms(fixed=mi, moving=pdlabels_img, transformlist=mytx['invtransforms'], interpolator='genericLabel')
    ants.image_write(warped_atlas, str(out_path / f'{exp_name}_pd25tor1_warped_atlas.nii'))
    if idx > 1:
        break

In [3]:
reg = ants.RegistrationSynQuick()
reg.inputs.fixed_image = str(mov_img_path)
reg.inputs.moving_image = subj_paths[0]
reg.inputs.output_prefix = 'ants_test'
reg.inputs.num_threads = 8
reg.cmdline

'antsRegistrationSyNQuick.sh -d 3 -f /mnt/projects/7TPD/bids/derivatives/hMRI_acu/mni_PD25_20170213_nifti/PD25-T1MPRAGE-template-1mm.nii -r 32 -m /mnt/projects/7TPD/bids/derivatives/hMRI_acu/derivatives/hMRI/sub-003/Results/brain_masked/sub-003_ses-01prisma3t_echo-01_part-magnitude-acq-MToff_MPM_R1_w.nii -n 8 -o ants_test -p d -s 26 -t s'

In [5]:
%%bash
reg.run()

bash: line 2: syntax error: unexpected end of file


CalledProcessError: Command 'b'reg.run()\n'' returned non-zero exit status 2.